## Napari basics 

## Why use Napari for DL workflows ?
* ROIs
* Layers
* Labelling tools

## Test image 


In [ ]:
from skimage.io import imread
import os 
import napari
import matplotlib.pyplot as plt
import numpy as np

data_path = r'./data'
parent_path = os.path.join(data_path, 'ladybugs_SAM')
img = imread(os.path.join(parent_path, '26638467_41374651.jpg'))
plt.imshow(img)

## Start Napari and add layers


In [ ]:
#| echo: true
#| eval: true 
viewer = napari.Viewer()

labels = np.zeros([img.shape[0], img.shape[1]], dtype=np.uint16)

viewer.add_image(img, name='ladybug')
viewer.add_labels(labels, name='labels')
rois_layer = viewer.add_shapes(face_color='transparent', edge_width=15, edge_color='blue', name='rois')

box = [[100,100], [100,800],[800,800], [800,100]]
rois_layer.add_rectangles(box)

## Add a cellpose layer


In [ ]:
#| echo: true
#| eval: true 
from cellpose import models, io
model_cyto3 = models.CellposeModel(gpu=True, model_type="cyto3")
results = np.zeros([img.shape[0], img.shape[1]], dtype=np.uint16)
viewer.add_labels(results, name='cyto3 masks')
results = model_cyto3.eval(img, diameter=140)
viewer.layers['cyto3 masks'].data = results[0]

In [ ]:
screenshot = viewer.screenshot(r'./napari_screenshot.png', canvas_only=False)
plt.imshow(screenshot)

## Get the ROI and labels we drew in Napari


In [ ]:
roi = viewer.layers['rois'].data
roi = np.array(roi[0], dtype=np.int16)

crop = img[ roi[0][0]:roi[2][0], roi[0][1]:roi[1][1]]
crop_label = labels[roi[0][0]:roi[2][0], roi[0][1]:roi[1][1]]
                    
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.imshow(crop)
plt.subplot(1,2,2)
plt.imshow(crop_label)
viewer.close()